<a href="https://colab.research.google.com/github/Esteban191/upgraded-goggles/blob/main/mysql_rese%C3%B1as_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from selenium import webdriver
from scrapy import Selector
import pandas as pd
import time
import sqlite3
import mysql.connector


In [ ]:

#ejecuto el drive de chrom que debe encontrarse en la ruta del archivo
chromedrive_path = './chromedriver' # use the path to the driver you downloaded from previous steps
driver = webdriver.Chrome(chromedrive_path)

<ipython-input-232-9ad4b2d43aa7>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [ ]:
#entro al url pro medio de seleinum 
url= "https://www.google.com/search?q=ortounino&rlz=1C5CHFA_enCO966CO966&ei=Q0g3ZKzsFqamqtsP_8i1yAI&ved=0ahUKEwjsmZjTyKX-AhUmk2oFHX9kDSkQ4dUDCA8&uact=5&oq=ortounino&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQAzIFCAAQogQyBQgAEKIEMgUIABCiBDIFCAAQogQyBQgAEKIEOgwIABCKBRBDEEYQ-QE6CwgAEIAEELEDEIMBOgUIABCABDoRCC4QgAQQsQMQgwEQxwEQ0QM6BwgAEIoFEEM6CAgAEIAEELEDOgsILhCABBCxAxCDAToFCC4QgAQ6CwguEIAEEMcBEK8BOg4IABCABBCxAxCDARDJAzoICAAQgAQQkgM6CAgAEIoFEJIDOhMILhCKBRDHARDRAxDUAhBDEOoEOg0IABCKBRCxAxCDARBDOh4ILhCKBRDHARDRAxDUAhBDEOoEENwEEN4EEOAEGAE6BwgAEIAEEBM6CQgAEIAEEAoQEzoGCAAQHhATOg0IABAFEB4Q8QQQChATOggIABAFEB4QEzoCCCY6DgguEIAEEMcBEK8BEOoEOgYIABAFEB46BAgAEB46GQguEIAEEMcBEK8BEOoEENwEEN4EEOAEGAFKBAhBGABQAFiFEGD7EmgBcAB4AIABmQKIAdwRkgEFMC4xLjmYAQCgAQHAAQHaAQYIARABGBQ&sclient=gws-wiz-serp#lrd=0x8e44284dcb8729cb:0x71931901ac163712,1"
driver.get(url)
time.sleep(5)

In [ ]:

#realizo scroll en el xpath de las reviws y reliza screoll en un while hara el final
import time
# Hacer scroll hasta el final de la página
element = driver.find_element_by_xpath("//div[@class='review-dialog-list']")
scroll_height = driver.execute_script("return arguments[0].scrollHeight", element)
last_scroll_height = 0

while last_scroll_height != scroll_height:
    last_scroll_height = scroll_height
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", element)
    time.sleep(1)
    scroll_height = driver.execute_script("return arguments[0].scrollHeight", element)


<ipython-input-234-153344c31b2e>:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element = driver.find_element_by_xpath("//div[@class='review-dialog-list']")


In [ ]:


page_content = driver.page_source
response = Selector(text=page_content)
results = []

for el in response.xpath('.//div[contains(@class , "gws-localreviews__google-review")]'):
    title = el.xpath('.//div[@class="TSUbDb"]/a/text()').extract_first('')
    body = el.xpath('.//span[@class="review-snippet" and not(@data-local-review-id)]/text() | .//div[@class="Jtu6Td"]/span/span/text()').extract_first('').strip()
    date = el.xpath('.//span[contains(text(),"Hace") or contains(text(),"semanas")]/text()').extract_first('')

    rating= el.xpath('.//span[@class = "lTi8oc z3HNkc"]/@aria-label').extract_first('').replace('stars' ,'').strip()
    #.//span[contains(@aria-label, "lTi8oc z3HNkc")]/@aria-label'

#agregar al array
    results.append({
        'title': title.strip() if title else '',
        'body': body.strip() if body else '',
        'date': date.strip() if date else '',
        "rating":rating.strip() if rating else " ",
        
    })
#crear datafremne
df = pd.DataFrame(results)

df['title'] = df['title'].str.upper()
df['body'] = df['body']
df['date'] = df['date']
df['rating'] = df['rating']


# Insretar registros a la base de datos de python por defecto que es sql3
conn = sqlite3.connect('./db/ortounion.db')
df.to_sql(name='reseñas', con=conn, if_exists='replace', index=False)
conn.commit()


In [ ]:
#df

In [ ]:

##conecccion a base de datos para mysql por xampaa
mydb = mysql.connector.connect(user='root', password='', database='odontologia', host='localhost', port=3307,charset='utf8mb4')
#print(mydb)


In [ ]:
#creacion de tabla mysql en xamppa
# Crear la tabla 'reseñas'
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE IF NOT EXISTS reseñas (id INT AUTO_INCREMENT PRIMARY KEY, title VARCHAR(255), body TEXT, date VARCHAR(255), rating VARCHAR(255))")

# Truncar la tabla antes de insertar nuevos registros
mycursor.execute("TRUNCATE TABLE reseñas")

# Iterar a través del DataFrame y agregar los registros a la tabla 'resenas'
for index, row in df.iterrows():
    title = row['title']
    body = row['body']
    date = row['date']
    rating = row['rating']
    
    # Insertar el registro en la tabla 'resenas'
    sql = "INSERT INTO reseñas (title, body, date, rating) VALUES (%s, %s, %s, %s)"
    val = (title, body, date, rating)
    mycursor.execute(sql, val)

# Guardar los cambios y cerrar la conexión a la base de datos
mydb.commit()
mydb.close()
